#### Imports

In [1]:
import sys
import os
import pandas as pd
import numpy as np
import re
import nltk

from keras.layers import Input, Embedding, LSTM, TimeDistributed, Dense, Bidirectional
from keras.models import Model, load_model

INPUT_LENGTH = 20
OUTPUT_LENGTH = 20


Using TensorFlow backend.


In [2]:
pirate_convert = pd.read_csv('pirate_convert.csv')
pirate_convert.head()

,L1045,They do not! Splice the mainbrace!
0,L1044,They do to! Cleave him to the brisket!
1,L985,I hope so. Avast ye!
2,L984,She okay? Arrr! Dead men tell no tales.
3,L925,Let's go. Thar she blows!
4,L924,Wow


In [3]:
len(pirate_convert)

304712

In [4]:
pirate_convert.columns=['id','dialog']
pirate_convert.head()

,id,dialog
0,L1044,They do to! Cleave him to the brisket!
1,L985,I hope so. Avast ye!
2,L984,She okay? Arrr! Dead men tell no tales.
3,L925,Let's go. Thar she blows!
4,L924,Wow


In [5]:
pirate_convert.append({'id':'L1045','dialog':'They do not! Splice the mainbrace!'}, ignore_index=True)

,id,dialog
0,L1044,They do to! Cleave him to the brisket!
1,L985,I hope so. Avast ye!
2,L984,She okay? Arrr! Dead men tell no tales.
3,L925,Let's go. Thar she blows!
4,L924,Wow
...,...,...
304708,L666370,I'm to pillage th' sikali with th' main column...
304709,L666369,"Your orders, mr vereker?"
304710,L666257,"Good ones, yes, mr vereker. Gentlemen who can ..."
304711,L666256,Colonel durnford... William vereker. I hear ye...


### Reading data

In [6]:
lines = open('cornell-moviedialog-corpus/movie_lines.txt', encoding='utf-8', errors='ignore').read().split('\n')
conv_lines = open('cornell-moviedialog-corpus/movie_conversations.txt', encoding='utf-8', errors='ignore').read().split('\n')

---
Sample of the data

---

### Extract the conversations for POC 1 from the dataset

In [8]:
# Create a dictionary to map each line's id with its text
id2line = {}

for i in range(pirate_convert.shape[0]):
    id2line[pirate_convert.iloc[i][0]] = pirate_convert.iloc[i][1]
    
    
    
#for line in lines:
#    _line = line.split(' +++$+++ ')
#    if len(_line) == 5:
#        if _line[2] == 'm159' :
#            id2line[_line[0]] = _line[4]

In [9]:
len(id2line)

304712

In [10]:
# Create a list of all of the conversations' lines' ids.
convs = []
for line in conv_lines[:-1]:
    _line = line.split(' +++$+++ ')
    if len(_line) >= 4:
        _line = _line[-1][1:-1].replace("'","").replace(" ","")
        convs.append(_line.split(','))

In [11]:
len(convs)

83097

### Sample of conversation

In [12]:
for k in convs[12]:
    print (k, id2line[k])

L404 She's not a... Blimey!
L405 Lesbian? No. I found a picture of jared leto in one of her drawers, so i'm pretty sure she's not harboring same-sex tendencies.
L406 So that's th' kind of guy she likes? Pretty ones?
L407 Who knows? All i've ever heard her say be that she'd dip before dating a guy that smokes.


---
Sort the sentences into questions (inputs) and answers (targets)

---

In [13]:
questions = []
answers = []
for conv in convs:
    for i in range(len(conv)-1):
        if conv[i+1] != 'L1045':
            questions.append(id2line[conv[i]])
            answers.append(id2line[conv[i+1]])
        
# Compare lengths of questions and answers
print(len(questions))
print(len(answers))

221615
221615


---
Text Cleaning

---

In [14]:
def clean_text(text):
    '''Clean text by removing unnecessary characters and altering the format of words.'''
    text = text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|]", "", text)
#     text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    text = " ".join(text.split())
    return text

In [15]:
# Clean the data
clean_questions = []
clean_answers = []    

for i in range(len(questions)):
    if type(questions[i]) == str and type(answers[i]) == str:
        clean_questions.append(clean_text(questions[i]))
        clean_answers.append(clean_text(answers[i]))

In [16]:
print(len(clean_questions))
print(len(clean_answers))



221281
221281


In [17]:
# Find the length of sentences (not using nltk due to processing speed)
lengths = []
# lengths.append([len(nltk.word_tokenize(sent)) for sent in clean_questions]) #nltk approach
for question in clean_questions:
    lengths.append(len(question.split()))
for answer in clean_answers:
    lengths.append(len(answer.split()))
# Create a dataframe so that the values can be inspected
lengths = pd.DataFrame(lengths, columns=['counts'])
print(np.percentile(lengths, 80))
print(np.percentile(lengths, 85))
print(np.percentile(lengths, 90))
print(np.percentile(lengths, 95))

18.0
21.0
26.0
35.0


In [18]:
# Remove questions and answers that are shorter than 1 word and longer than 20 words.
min_line_length = 1
max_line_length = 20

# Filter out the questions that are too short/long
short_questions_temp = []
short_answers_temp = []

for i, question in enumerate(clean_questions):
    if len(question.split()) >= min_line_length and len(question.split()) <= max_line_length:
        short_questions_temp.append(question)
        short_answers_temp.append(clean_answers[i])

# Filter out the answers that are too short/long
short_questions = []
short_answers = []

for i, answer in enumerate(short_answers_temp):
    if len(answer.split()) >= min_line_length and len(answer.split()) <= max_line_length:
        short_answers.append(answer)
        short_questions.append(short_questions_temp[i])
        
print(len(short_questions))
print(len(short_answers))

157278
157278


### Sample question-answer pairs

In [19]:
r = np.random.randint(1,len(short_questions))

for i in range(r, r+3):
    print(short_questions[i])
    print(short_answers[i])
    print()

i came all th' way here for that? to walk th' lobby?
yeah. and it might have even worked too.

yeah. and it might have even worked too.
let's do it again. blimey!

ye believe they are shooting a nike ad down there? did i ever tell ye me nike story?
i gotta get back to cushman.



### Train-Test Validation

In [20]:
#choosing number of samples
num_samples = 30000  # Number of samples to train on.
short_questions = short_questions[:num_samples]
short_answers = short_answers[:num_samples]
#tokenizing the qns and answers
short_questions_tok = [nltk.word_tokenize(sent) for sent in short_questions]
short_answers_tok = [nltk.word_tokenize(sent) for sent in short_answers]

In [21]:
 len(short_questions_tok)

30000

In [22]:
#train-validation split
data_size = len(short_questions_tok)

# We will use the first 0-80th %-tile (80%) of data for the training
training_input  = short_questions_tok[:round(data_size*(80/100))]
training_input  = [tr_input[::-1] for tr_input in training_input] #reverseing input seq for better performance
training_output = short_answers_tok[:round(data_size*(80/100))]

# We will use the remaining for validation
validation_input = short_questions_tok[round(data_size*(80/100)):]
validation_input  = [val_input[::-1] for val_input in validation_input] #reverseing input seq for better performance
validation_output = short_answers_tok[round(data_size*(80/100)):]

print('training size', len(training_input))
print('validation size', len(validation_input))

training size 24000
validation size 6000


### Word en/decoding dictionaries

In [23]:
# Create a dictionary for the frequency of the vocabulary
# Create 
vocab = {}
for question in short_questions_tok:
    for word in question:
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1

for answer in short_answers_tok:
    for word in answer:
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1 

In [24]:
# Remove rare words from the vocabulary.
# We will aim to replace fewer than 5% of words with <UNK>
# You will see this ratio soon.
threshold = 10
count = 0
for k,v in vocab.items():
    if v >= threshold:
        count += 1
        
        
print("Size of total vocab:", len(vocab))
print("Size of vocab we will use:", count)        

Size of total vocab: 15213
Size of vocab we will use: 2519


In [25]:
#we will create dictionaries to provide a unique integer for each word.
WORD_CODE_START = 1
WORD_CODE_PADDING = 0


word_num  = 2 #number 1 is left for WORD_CODE_START for model decoder later
encoding = {}
decoding = {1: 'START'}
for word, count in vocab.items():
    if count >= threshold: #get vocabularies that appear above threshold count
        encoding[word] = word_num 
        decoding[word_num ] = word
        word_num += 1

print("No. of vocab used:", word_num)


No. of vocab used: 2521


In [26]:
#include unknown token for words not in dictionary
decoding[len(encoding)+2] = '<UNK>'
encoding['<UNK>'] = len(encoding)+2

In [27]:
dict_size = word_num+1
dict_size

2522

### Vectorization

In [28]:
def transform(encoding, data, vector_size=20):
    """
    :param encoding: encoding dict built by build_word_encoding()
    :param data: list of strings
    :param vector_size: size of each encoded vector
    """
    transformed_data = np.zeros(shape=(len(data), vector_size))
    for i in range(len(data)):
        for j in range(min(len(data[i]), vector_size)):
            try:
                transformed_data[i][j] = encoding[data[i][j]]
            except:
                transformed_data[i][j] = encoding['<UNK>']
    return transformed_data

In [29]:
#encoding training set
encoded_training_input = transform(
    encoding, training_input, vector_size=INPUT_LENGTH)
encoded_training_output = transform(
    encoding, training_output, vector_size=OUTPUT_LENGTH)

print('encoded_training_input', encoded_training_input.shape)
print('encoded_training_output', encoded_training_output.shape)


encoded_training_input (24000, 20)
encoded_training_output (24000, 20)


In [30]:
#encoding validation set
encoded_validation_input = transform(
    encoding, validation_input, vector_size=INPUT_LENGTH)
encoded_validation_output = transform(
    encoding, validation_output, vector_size=OUTPUT_LENGTH)

print('encoded_validation_input', encoded_validation_input.shape)
print('encoded_validation_output', encoded_validation_output.shape)

encoded_validation_input (6000, 20)
encoded_validation_output (6000, 20)


# Model Building

### Sequence-to-Sequence in Keras

In [31]:
import tensorflow as tf
tf.keras.backend.clear_session()

In [32]:
INPUT_LENGTH = 20
OUTPUT_LENGTH = 20

encoder_input = Input(shape=(INPUT_LENGTH,))
decoder_input = Input(shape=(OUTPUT_LENGTH,))

In [33]:
from keras.layers import SimpleRNN

encoder = Embedding(dict_size, 128, input_length=INPUT_LENGTH, mask_zero=True)(encoder_input)
encoder = LSTM(512, return_sequences=True, unroll=True)(encoder)
encoder_last = encoder[:,-1,:]

print('encoder', encoder)
print('encoder_last', encoder_last)

decoder = Embedding(dict_size, 128, input_length=OUTPUT_LENGTH, mask_zero=True)(decoder_input)
decoder = LSTM(512, return_sequences=True, unroll=True)(decoder, initial_state=[encoder_last, encoder_last])

print('decoder', decoder)

# For the plain Sequence-to-Sequence, we produced the output from directly from decoder
# output = TimeDistributed(Dense(output_dict_size, activation="softmax"))(decoder)

encoder Tensor("lstm_1/transpose_2:0", shape=(None, 20, 512), dtype=float32)
encoder_last Tensor("strided_slice:0", shape=(None, 512), dtype=float32)
decoder Tensor("lstm_2/transpose_2:0", shape=(None, 20, 512), dtype=float32)


## Attention Mechanism

In [34]:
from keras.layers import Activation, dot, concatenate

# Equation (7) with 'dot' score from Section 3.1 in the paper.
# Note that we reuse Softmax-activation layer instead of writing tensor calculation
attention = dot([decoder, encoder], axes=[2, 2])
attention = Activation('softmax', name='attention')(attention)
print('attention', attention)

context = dot([attention, encoder], axes=[2,1])
print('context', context)

decoder_combined_context = concatenate([context, decoder])
print('decoder_combined_context', decoder_combined_context)

# Has another weight + tanh layer as described in equation (5) of the paper
output = TimeDistributed(Dense(512, activation="tanh"))(decoder_combined_context)
output = TimeDistributed(Dense(dict_size, activation="softmax"))(output)
print('output', output)

attention Tensor("attention/truediv:0", shape=(None, 20, 20), dtype=float32)
context Tensor("dot_2/MatMul:0", shape=(None, 20, 512), dtype=float32)
decoder_combined_context Tensor("concatenate_1/concat:0", shape=(None, 20, 1024), dtype=float32)
output Tensor("time_distributed_2/Reshape_1:0", shape=(None, 20, 2522), dtype=float32)


In [35]:
model = Model(inputs=[encoder_input, decoder_input], outputs=[output])
model.compile(optimizer='adam', loss='binary_crossentropy')
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 20, 128)      322816      input_2[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 20, 128)      322816      input_1[0][0]                    
____________________________________________________________________________________________

In [36]:
training_encoder_input = encoded_training_input
training_decoder_input = np.zeros_like(encoded_training_output)
training_decoder_input[:, 1:] = encoded_training_output[:,:-1]
training_decoder_input[:, 0] = WORD_CODE_START
training_decoder_output = np.eye(dict_size)[encoded_training_output.astype('int')]

validation_encoder_input = encoded_validation_input
validation_decoder_input = np.zeros_like(encoded_validation_output)
validation_decoder_input[:, 1:] = encoded_validation_output[:,:-1]
validation_decoder_input[:, 0] = WORD_CODE_START
validation_decoder_output = np.eye(dict_size)[encoded_validation_output.astype('int')]

In [37]:
model.fit(x=[training_encoder_input, training_decoder_input], y=[training_decoder_output],
          validation_data=([validation_encoder_input, validation_decoder_input], [validation_decoder_output]),
          #validation_split=0.05,
          batch_size=64, epochs=100)

model.save('model_attention_20.h5')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 24000 samples, validate on 6000 samples
Epoch 1/100
24000/24000 [==============================] - 393s 16ms/step - loss: 0.0016 - val_loss: 0.0014
Epoch 2/100
24000/24000 [==============================] - 391s 16ms/step - loss: 0.0012 - val_loss: 0.0011
Epoch 3/100
24000/24000 [==============================] - 393s 16ms/step - loss: 0.0011 - val_loss: 0.0010
Epoch 4/100
24000/24000 [==============================] - 391s 16ms/step - loss: 0.0010 - val_loss: 9.9031e-04
Epoch 5/100
24000/24000 [==============================] - 393s 16ms/step - loss: 9.8784e-04 - val_loss: 9.7107e-04
Epoch 6/100
24000/24000 [==============================] - 394s 16ms/step - loss: 9.6732e-04 - val_loss: 9.5471e-04
Epoch 7/100
24000/24000 [==============================] - 397s 17ms/step - loss: 9.5201e-04 - val_loss: 9.4777e-04
Epoch 8/100
24000/24000 [==============================] - 399s 17ms/step - loss: 9.3948e-04 - val_loss: 9.3933e-04
Epoch 9/100
24000/24000 [==========================

24000/24000 [==============================] - 385s 16ms/step - loss: 2.9349e-04 - val_loss: 0.0015
Epoch 73/100
24000/24000 [==============================] - 386s 16ms/step - loss: 2.8769e-04 - val_loss: 0.0015
Epoch 74/100
24000/24000 [==============================] - 387s 16ms/step - loss: 2.8177e-04 - val_loss: 0.0015
Epoch 75/100
24000/24000 [==============================] - 387s 16ms/step - loss: 2.7351e-04 - val_loss: 0.0015
Epoch 76/100
24000/24000 [==============================] - 385s 16ms/step - loss: 2.6908e-04 - val_loss: 0.0015
Epoch 77/100
24000/24000 [==============================] - 386s 16ms/step - loss: 2.6290e-04 - val_loss: 0.0015
Epoch 78/100
24000/24000 [==============================] - 386s 16ms/step - loss: 2.6014e-04 - val_loss: 0.0015
Epoch 79/100
24000/24000 [==============================] - 385s 16ms/step - loss: 2.6129e-04 - val_loss: 0.0015
Epoch 80/100
24000/24000 [==============================] - 385s 16ms/step - loss: 2.6941e-04 - val_loss: 0.0

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/engine/network.py:896: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'strided_slice:0' shape=(None, 512) dtype=float32>, <tf.Tensor 'strided_slice:0' shape=(None, 512) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


##### Model Testing

In [38]:
def prediction(raw_input):
    clean_input = clean_text(raw_input)
    input_tok = [nltk.word_tokenize(clean_input)]
    input_tok = [input_tok[0][::-1]]  #reverseing input seq
    encoder_input = transform(encoding, input_tok, 20)
    decoder_input = np.zeros(shape=(len(encoder_input), OUTPUT_LENGTH))
    decoder_input[:,0] = WORD_CODE_START
    for i in range(1, OUTPUT_LENGTH):
        output = model.predict([encoder_input, decoder_input]).argmax(axis=2)
        decoder_input[:,i] = output[:,i]
    return output

def decode(decoding, vector):
    """
    :param decoding: decoding dict built by word encoding
    :param vector: an encoded vector
    """
    text = ''
    for i in vector:
        if i == 0:
            break
        text += ' '
        text += decoding[i]
    return text

In [44]:
for i in range(5):
    seq_index = np.random.randint(1, len(short_questions))
    output = prediction(short_questions[seq_index])
    print ('Q:', short_questions[seq_index])
    print ('A:', decode(decoding, output[0]),'\n')


Q: oh, you have lost it, have not ya? dead men tell no tales.
A:  i am here . thar she blows ! 

Q: were those flowers really for me, brad?
A:  i do not want to pillage ye out with him . 

Q: rae... savvy? savvy?
A:  i do not see why we got ta lie about it when ye and i know this ai not not 

Q: three, four years. avast ye!
A:  that is all right now ? avast ye ! 

Q: where? savvy?
A:  we are in th ' <UNK> of th ' <UNK> of <UNK> . 



In [40]:
texts = ["Who are you?", "How are you doing?", "What is your purpose?", "What can you do?", "I need help", "Can you help me?"]

for text in texts:
    output = prediction(text)
    print ('Q:', text)
    print ('A:', decode(decoding, output[0]))
    print()

Q: Who are you?
A:  in th ' bedroom .

Q: How are you doing?
A:  not here 's than i get in !

Q: What is your purpose?
A:  whar be th ' emperor and th ' army , soldier ?

Q: What can you do?
A:  i think you are a freak . i think ye do this to <UNK> me . and i think think

Q: I need help
A:  i know . arrr !

Q: Can you help me?
A:  i will get him . splice the mainbrace !

